In [2]:
from importlib import reload #by ChatGPT

In [18]:
#by ChatGPT
import toy_sklearn  
reload(toy_sklearn)

<module 'toy_sklearn' from '/Users/morikawakaion/Desktop/self_studies/my_ml_tools/toy_sklearn.py'>

In [13]:
lr = toy_sklearn.linear_model.LinearRegression()

In [19]:
x = toy_sklearn.model_selection.train_test_split(data=1, test_size=0.2)

In [20]:
x

2